https://edinburghcyclehire.com/
Souřadnnice sanic jsou v Decimal degrees in WGS84. Pro použití na openstreet.org mapách jsou přepočteny na pseudo Mercator projekci.



### Příprava dat:
#### Identifikace chybných hodnot 
Zdrojová data obsahují nekonzistence v názvech a popisech stanic. Jejich zeměpisná šířka a délka také není u všech údajů stejná (odchylka GPS).\
Odstraněno seskupením stanic dle jejich id a výpočtem průměru souřadnic.


#### Redukce počtu proměnných
##### Blízké klastry
- sloučením stanic vzdálených od sebe méně než 25 metrů se sníží počet stanic o 45 (22,5%). Odstraněno přepsáním v df rides, vymazáním ze df stations.

##### Vzdálené klastry
- odstraněním stanic vzdálených od sebe více než 50 kilometrů se sníží počet stanic o 1 (Liverpool) - optimalizujeme následný výpočet klastrů. Odstraněno vymazáním v df rides a df stations.  

##### Málo četné a dočasné klastry
 - odstraněním stanic použitých méně než 10 x (pro stanice starší 90 dnů) se sníží počet stanic o 8 (4%). Odstraněno vymazáním v df rides a df stations.
 - odstraněním dočasných stanic ("19th to 23rd June", "festival" a "event") se sníží počet stanic o 6 (3%). Odstraněno vymazáním v df rides a df stations.

Celkem se podařilo snížit množství stanic pro následnou analýzu o 60 t.j. 30 %.




lze načíst výšku stanic z openstreetmap.org ?
k-means

První krok modelování - závěry z obvykle nesupervizovaného modelu jsou vstupem pro následné supervizované modelování
Identifikace podezřelých případů 



In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
from bokeh.io import output_notebook, show
from bokeh.models import WMTSTileSource
from bokeh.plotting import figure, ColumnDataSource
from sklearn.neighbors import KDTree
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 500)
pd.set_option("display.precision", 14)
output_notebook()

In [ ]:
# vyloučit 3 "jízdy" s end stanicí 280 (Exhibition Centre Liverpool)
rides = pd.read_csv("201809-202010.csv", parse_dates=[0,1], usecols = ["started_at","ended_at","duration","start_station_id", "end_station_id"])    

In [ ]:
# Vytvoření tabulky obsahující stanice = načtení z csv 
# Vymazána ? koncová stanice id=280 iloc=30 - Exhibition Centre Liverpool (výstava kol).
# Celková tabulka stations je vytvořena spojením informací ze "start" a "end" slouců. Duplicity jsou řešeny vybráním prvního popisu a průměrem souřadnic - vše dle id.
# Další nekonzistencí je existence více stanic na jednom místě. Vyberu ty, které mají více použití.

# Načtení informací o "start" a "end" stanicích:
stations = pd.read_csv("201809-202010.csv", index_col = "start_station_id", usecols = ["start_station_id","start_station_name","start_station_description","start_station_latitude", "start_station_longitude"])   
end_stations = pd.read_csv("201809-202010.csv", index_col = "end_station_id", usecols = ["end_station_id","end_station_name","end_station_description","end_station_latitude", "end_station_longitude"])  
# Přejmenování sloupců a indexu:
column_names = {"start_station_name":"name","start_station_description":"description","start_station_latitude":"latitude", "start_station_longitude":"longitude"}
stations.rename(columns=column_names, index={"start_station_id": "idd"}, inplace=True)
stations.index.names = ['id']
end_column_names = {"end_station_name":"name","end_station_description":"description","end_station_latitude":"latitude", "end_station_longitude":"longitude"}
end_stations.rename(columns=end_column_names, index={'end_station_id': 'id'}, inplace=True)
end_stations.index.names = ['id']
# Sloučení informací o "start" a "end" stanicích a jejich setřídění:
stations = stations.append(end_stations)
stations.sort_index(axis="index", inplace = True)
# Výpočet průměrných koorinátů a přidání k prvním názvům a popisům stanic:
stations_coord_mean = stations.groupby(["id"]).mean()
stations = stations.groupby(["id"]).first()
stations["latitude"] = stations_coord_mean["latitude"]
stations["longitude"] = stations_coord_mean["longitude"]
stations["description"].fillna("", inplace=True)
# Vymazána koncová stanice id 280 - Exhibition Centre Liverpool (výstava kol). Lepším řešením je vymazat "stanice", které jsou vzdáleny od Edinburgu víc než 50? km.
# stations.drop(280, inplace=True)
# přepočet wgs84 souřadnic na  Mercator projekci pro použití v WMTS tile map v openstreetmap.org 
stations["mer_x"] = stations["longitude"] * (6378137 * np.pi/180.0)
stations["mer_y"] = np.log(np.tan((90 + stations["latitude"]) * np.pi/360.0)) * 6378137
# Vymazání dočasných df:
end_stations = end_stations[0:0]
stations_coord_mean = stations_coord_mean[0:0]

In [ ]:
# Výpočet počtu použití stanic (počítá se půjčení a vrácení zvlášť, v případě půjčení a vrácení na stejnou stanici se počítá 2):
stations["total_rides"] = rides.groupby(["start_station_id"])[["duration"]].count()
stations["total_rides"].fillna(0, inplace = True)
stations["total_rides_tmp"] = rides.groupby(["end_station_id"])[["duration"]].count()
stations["total_rides_tmp"].fillna(0, inplace = True)
stations["total_rides"] = stations["total_rides"].astype(int) + stations["total_rides_tmp"].astype(int)
stations["rides"] = stations["total_rides"]
stations.drop("total_rides_tmp", axis=1, inplace=True)
# Výpočet celkového použití stanic
rides_end = rides.groupby(["end_station_id"])["end_station_id"].count()
rides_start = rides.groupby(["start_station_id"])["start_station_id"].count()
idx = rides_end.index.append(rides_start.index).unique()
balance = pd.DataFrame(index=idx)
balance["end"] = rides_end 
balance["start"] = rides_start
balance.fillna(0, inplace=True)
stations["total_balance"] = balance["end"] - balance["start"]
stations["balance"] = stations["total_balance"]

In [ ]:
# Vyhledání nejbližších sousedů v poloměru 50 metrů a jejich sloučení v df stations a rides: KD Tree (median x rozdělí body do 2 skupin, pak medián y do 4 skupin, atd.)
# Spouštět dokud nachází duplicity
X = stations[["mer_x", "mer_y"]].values
tree = KDTree(X, leaf_size=40, metric='euclidean')     
ind = tree.query_radius(X, r=50, return_distance=True, sort_results=True)  # indices of said neighbors
duplicity = []
for station in range(len(ind[0])):
    try:
        duplicity.append([ind[0][station][0], ind[0][station][1]]) # , round(ind[1][station][1],1)
    except:
        pass
print(f"{len(duplicity)} párů souřadnic, t.j {len(duplicity)/2} duplicit.")

rows_for_deletion = []
dict_for_substitute = {}
for station in duplicity:
    if station[0] < station[1]:
        row = stations.iloc[station[0]].name
        row_for_deletion = stations.iloc[station[1]].name
        dict_for_substitute[row_for_deletion] = row
        stations.at[row, "mer_x"] = (stations.at[row, "mer_x"] + stations.at[row_for_deletion, "mer_x"])/2
        stations.at[row, "mer_y"] = (stations.at[row, "mer_y"] + stations.at[row_for_deletion, "mer_y"])/2
        stations.at[row, "latitude"] = (stations.at[row, "latitude"] + stations.at[row_for_deletion, "latitude"])/2
        stations.at[row, "longitude"] = (stations.at[row, "longitude"] + stations.at[row_for_deletion, "longitude"])/2
        stations.at[row, "total_rides"] = stations.at[row, "total_rides"] + stations.at[row_for_deletion, "total_rides"]
        rows_for_deletion.append(row_for_deletion)

stations.drop(index=rows_for_deletion, axis = 0, inplace=True)
rides["start_station_id"].replace(to_replace=dict_for_substitute , inplace=True)
rides["end_station_id"].replace(to_replace=dict_for_substitute , inplace=True)

In [ ]:
# Výpočet salda a počtu jízd v zadaném intervalu
day_start = pd.Timestamp("2020-01-01", unit = "ns", tz="UTC")
day_end = pd.Timestamp("2020-01-31T235959", unit = "ns", tz="UTC")
mask = (day_start <= rides["started_at"]) & (rides["ended_at"] <= day_end)
rides_end = rides.loc[mask].groupby(["end_station_id"])["end_station_id"].count()
rides_start = rides.loc[mask].groupby(["start_station_id"])["start_station_id"].count()
idx = rides_end.index.append(rides_start.index).unique()
balance = pd.DataFrame(index=idx)
balance["end"] = rides_end 
balance["start"] = rides_start
balance.fillna(0, inplace=True)
stations["balance"] = balance["end"] - balance["start"]
stations["rides"] = balance["end"] + balance["start"]
balance = [0,0]

In [ ]:
# k-means clustering
from numpy import unique
from numpy import where
from sklearn.datasets import make_classification
from sklearn.cluster import KMeans
from matplotlib import pyplot

TOOLTIPS = [("id", "@{id}"), ("(x,y)", "(@{latitude}, @{longitude})"), ("name", "@{name}"), ("description", "@{description}"), ("total rides", "@{total_rides}"),
            ("total balance", "@{total_balance}"), ("selected rides", "@{rides}"),("selected balance", "@{balance}"),]
p = figure(plot_width=1200, plot_height=700, tools='reset, pan, wheel_zoom, save, box_zoom', active_scroll="wheel_zoom", x_range=(-380000,-340000), y_range=(7545000,7553000), 
           x_axis_type="mercator", y_axis_type="mercator", tooltips=TOOLTIPS, title="Edinburg - Just Eat Cycles") # Edinburg
p.add_tile(WMTSTileSource(url="https://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"))

# define dataset
X = stations[["mer_x", "mer_y"]].values.tolist()
W = stations["total_rides"].values.tolist()
#X, _ = make_classification(n_samples=100, n_features=2, n_informative=2, n_redundant=0, n_clusters_per_class=1, random_state=4)
# define the model
model = KMeans(n_clusters=20)
# fit the model
model.fit(X, sample_weight=W)
# assign a cluster to each example
yhat = model.predict(X, sample_weight=W)
# retrieve unique clusters
clusters = unique(yhat)
# create scatter plot for samples from each cluster
for cluster in clusters:
    # get row indexes for samples with this cluster
    row_ix = where(yhat == cluster)
    row_ix = row_ix[0].tolist()
    # create scatter of these samples
#    pyplot.scatter(X[row_ix[0]][0], X[row_ix[0]][1])
    p.scatter(X[row_ix[0]][0], X[row_ix[0]][1], size=W[row_ix[0]]/100, color="blue", alpha=0.3)    # vybírám pouze část sloupců, protože bokeh nemůže mít názvy sloupců čísla !!!
    p.scatter(X[row_ix[0]][0], X[row_ix[0]][1], radius=300, color="yellow", alpha=1)

p.circle("mer_x", "mer_y",  radius=100, color="red", line_width=.8, line_color='black', alpha=0.5, source=ColumnDataSource(data=stations))    
show(p)

#pyplot.show()

In [ ]:
stations.head()

In [ ]:
TOOLTIPS = [("id", "@{id}"), ("(x,y)", "(@{latitude}, @{longitude})"), ("name", "@{name}"), ("description", "@{description}"), ("total rides", "@{total_rides}"),
            ("total balance", "@{total_balance}"), ("selected rides", "@{rides}"),("selected balance", "@{balance}"),]
start = day_start.strftime("%Y-%m-%d")
end = day_end.strftime("%Y-%m-%d")
title = f"Edinburg - Just Eat Cycles - selected balance from {start} to {end}"

p = figure(plot_width=1200, plot_height=700, tools='reset, pan, wheel_zoom, save, box_zoom', active_scroll="wheel_zoom", x_range=(-380000,-340000), y_range=(7545000,7553000), 
           x_axis_type="mercator", y_axis_type="mercator", tooltips=TOOLTIPS, title=title) # Edinburg
p.add_tile(WMTSTileSource(url="https://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"))

#p.circle("mer_x", "mer_y",  radius=100, color="red", line_width=.8, line_color='black', alpha=0.5, source=ColumnDataSource(data=stations))     # bokeh nemůže mít názvy sloupců čísla !!!
p.circle("mer_x", "mer_y", radius="balance", color="purple", alpha=0.8, source=ColumnDataSource(data=stations.loc[stations["balance"]<0]))    #  bokeh nemůže mít názvy sloupců čísla !!!
p.circle("mer_x", "mer_y", radius="balance", color="blue", alpha=0.6, source=ColumnDataSource(data=stations.loc[stations["balance"]>0]))    #  bokeh nemůže mít názvy sloupců čísla !!!

show(p)

In [ ]:
# openstreetmap používá mercator coordináty CIRCLE
TOOLTIPS = [("id", "@{id}"), ("(x,y)", "(@{latitude}, @{longitude})"), ("name", "@{name}"), ("description", "@{description}"), ("total rides", "@{total_rides}"),
            ("total balance", "@{total_balance}"), ("selected rides", "@{rides}"),("selected balance", "@{balance}"),]
p = figure(plot_width=1200, plot_height=700, tools='reset, pan, wheel_zoom, save, box_zoom', active_scroll="wheel_zoom", x_range=(-380000,-340000), y_range=(7545000,7553000), 
           x_axis_type="mercator", y_axis_type="mercator", tooltips=TOOLTIPS, title="Edinburg - Just Eat Cycles - stations") # Edinburg
p.add_tile(WMTSTileSource(url="https://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"))

p.circle("mer_x", "mer_y",  radius=100, color="red", line_width=.8, line_color='black', alpha=0.5, source=ColumnDataSource(data=stations))    # bokeh nemůže mít názvy sloupců čísla !!!
show(p)